# Train model on colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kk17/supermark_det/blob/main/notebooks/training_on_cloud.ipynb)


In [7]:
%%bash
# clone project repo
git clone https://github.com/kk17/supermark_det
cd supermark_det
./init.sh


  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
Cloning https://github.com/tensorflow/models
Installing protobuf
Found protoc
Install the Object Detection API
Processing /content/supermark_det/tensorflow_model_garden/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1667808 sha256=bb448f4667b9ce8d3c73a292d5a292750df01195b58ca1fe93b3449be6146e29
  Stored in directory: /tmp/pip-ephem-wheel-cache-ohr296d_/wheels/04/fc/25/3a78d67772a8c692e6da7c236d33cbc9f28ec6bea46c1e75fd
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22258 sha256=92dd1c8ca37ee069e3785ee623793c91465791bcbd8c7c1b745289115d6ddb84
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/0

fatal: destination path 'supermark_det' already exists and is not an empty directory.
+ XTRACE=false
+ [[ false = \t\r\u\e ]]
+ IFS='
	'
+++ dirname ./init.sh
++ cd .
++ pwd
+ DIR=/content/supermark_det
+ pip install -r requirements.txt
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
+ TENSORFLOW_MODEL_GARDEN_DIR=/content/supermark_det/tensorflow_model_garden
+ WORKSPACE_DIR=/content/supermark_det/workspace
+ SKIP_CLONE=false
+ getopts s FLAG
+ command -v git
+ '[' false = false ']'
+ '[' -d /content/supermark_det/tensorflow_model_garden ']'
+ echo 'Cloning https://github.com/tensorflow/models'
+ git clone --depth 1 https://github.com/tensorflow/models /content/supermark_det/tensorflow_model_garden
Cloning into '/content/supermark_det/tensorflow_model_garden'...
+ cd /co

In [4]:
pwd

'/content'

In [8]:
# check installation
import tensorflow.compat.v2 as tf
from google.protobuf import text_format
from object_detection import exporter_lib_v2
from object_detection.protos import pipeline_pb2

## mount drive

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
!rsync -r /content/drive/MyDrive/supermarket_detection_workspace/ /content/supermark_det/workspace

## Use colabcode

In [15]:
!pip install colabcode

     |████████████████████████████████| 8.3 MB 7.7 MB/s 
     |████████████████████████████████| 745 kB 50.2 MB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 
     |████████████████████████████████| 58 kB 5.6 MB/s 
     |████████████████████████████████| 428 kB 37.5 MB/s 
     |████████████████████████████████| 393 kB 58.0 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 125 kB 59.8 MB/s 
     |████████████████████████████████| 124 kB 59.4 MB/s 
     |████████████████████████████████| 124 kB 61.8 MB/s 
     |████████████████████████████████| 122 kB 60.3 MB/s 
     |████████████████████████████████| 122 kB 64.1 MB/s 
     |████████████████████████████████| 122 kB 62.0 MB/s 
     |████████████████████████████████| 122 kB 59.6 MB/s 
     |████████████████████████████████| 112 kB 61.6 MB/s 
     |████████████████████████████████| 69 kB 7.0 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 s

In [ ]:
from colabcode import ColabCode

# vscode
ColabCode(port=10000)
# jupyter lab
# ColabCode(port=10000, lab=True)

Code Server can be accessed on: NgrokTunnel: "https://9d2d-34-71-201-237.ngrok.io" -> "http://localhost:10000"
[2021-10-13T07:20:23.304Z] info  code-server 3.10.2 387b12ef4ca404ffd39d84834e1f0776e9e3c005
[2021-10-13T07:20:23.306Z] info  Using user-data-dir ~/.local/share/code-server
[2021-10-13T07:20:23.329Z] info  Using config file ~/.config/code-server/config.yaml
[2021-10-13T07:20:23.329Z] info  HTTP server listening on http://127.0.0.1:10000 
[2021-10-13T07:20:23.329Z] info    - Authentication is disabled 
[2021-10-13T07:20:23.329Z] info    - Not serving HTTPS 


# install fiftyone

[FiftyOne](https://voxel51.com/docs/fiftyone/index.html) is open-source tool for building high-quality datasets and computer vision models.


In [ ]:
!pip install fiftyone


In [118]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone.core import dataset as D
from fiftyone import ViewField as F
import tensorflow as tf

import os


# Functions to build customized dataset


In [29]:
classes = ["apple", "orange", "banana", "book"]
oi_classes = [c[0].upper() + c[1:] for c in classes]

In [56]:
def build_dataset(
    classes,
    source_dataset='open-images-v6',
    split="validation",
    label_types=['detections'],
    max_samples_per_class=10,
    delete_exist_dataset=False
    ):

    all_calss_dataset_name = f'{source_dataset}-{split}-{max_samples_per_class}'
    dataset = None
    if D.dataset_exists(all_calss_dataset_name):
        if delete_exist_dataset:
            print(f'Delete existing dataset {all_calss_dataset_name}')
            D.delete_dataset(all_calss_dataset_name)
        else:
            print(f'Load existing dataset {all_calss_dataset_name}')
            dataset = D.load_dataset(all_calss_dataset_name)
    if dataset:
        return dataset

    single_class_datasets = []
    for c in classes:
        dataset_name = f'{source_dataset}-{split}-{c}-{max_samples_per_class}'
        single_class_dataset = None
        if D.dataset_exists(dataset_name):
            if delete_exist_dataset:
                print(f'Delete existing dataset {dataset_name}')
                D.delete_dataset(dataset_name)
            else:
                print(f'Load existing dataset {dataset_name}')
                single_class_dataset = D.load_dataset(dataset_name)

        if not single_class_dataset:
            single_class_dataset = foz.load_zoo_dataset(
                source_dataset,
                split=split,
                label_types=label_types,
                classes=[c], 
                max_samples=max_samples_per_class,
                dataset_name=dataset_name
            )
        single_class_datasets.append(single_class_dataset)

    dataset = single_class_datasets[0].clone(all_calss_dataset_name)
    for d in single_class_datasets[1:]:
        dataset.merge_samples(d)
    return dataset


In [104]:
# Bboxes are in [top-left-x, top-left-y, width, height] format
bbox_area = F("bounding_box")[2] * F("bounding_box")[3]


def build_dateset_view(dataset,
                       dataset_type='open_image',
                       IsOccluded=None,
                       IsTruncated=None,
                       IsGroupOf=None,
                       IsDepiction=None,
                       IsInside=None,
                       iscrowd=None,
                       valid_labels=oi_classes,
                       bbox_area_lower_bound=None):
    if dataset_type =='open_image':
        gt_field = 'detections'
    else:
        gt_field = 'ground_truth'
    view = dataset
    if valid_labels:
        view = view.filter_labels(gt_field,
                                  F("label").is_in(valid_labels),
                                  only_matches=False)
    if IsOccluded is not None:
        view = view.filter_labels(gt_field,
                                  F("IsOccluded") == IsOccluded,
                                  only_matches=True)
    if IsTruncated is not None:
        view = view.filter_labels(gt_field,
                                  F("IsTruncated") == IsTruncated,
                                  only_matches=True)
    if IsGroupOf is not None:
        view = view.filter_labels(gt_field,
                                  F("IsGroupOf") == IsGroupOf,
                                  only_matches=True)
    if IsDepiction is not None:
        view = view.filter_labels(gt_field,
                                  F("IsDepiction") == IsDepiction,
                                  only_matches=True)
    if IsInside is not None:
        view = view.filter_labels(gt_field,
                                  F("IsInside") == IsInside,
                                  only_matches=True)
    if iscrowd is not None:
        view = view.filter_labels(gt_field,
                                  F("iscrowd") == iscrowd,
                                  only_matches=True)
    if bbox_area_lower_bound:
        view = view.filter_labels(
            gt_field,
            bbox_area > bbox_area_lower_bound,
        )
    return view


In [114]:
def export_dataset_or_view(
    dataset_or_view,
    export_dir,
    label_field,
    dataset_type=fo.types.COCODetectionDataset,
    overwrite=False,
    tf_file_name=None,
):
    dataset_or_view.export(
        export_dir=export_dir,
        dataset_type=dataset_type,
        label_field=label_field,
        overwrite=overwrite
    )

    if dataset_type == fo.types.TFObjectDetectionDataset and tf_file_name:
        if not tf_file_name.endswith('.records'):
            tf_file_name += '.records'
        source = os.path.join(export_dir, 'tf.records')
        target = os.path.join(export_dir, tf_file_name)
        if os.path.exists(target) and overwrite:
            os.remove(target)
        os.rename(source, target)

In [125]:
def check_tf_record(filepath, take_num=1):
    raw_dataset = tf.data.TFRecordDataset(filepath)
    for raw_record in raw_dataset.take(take_num):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        print(example)
        # print('-' * 32)
        # n = len(example.features)
        # for i in range(n):
        #     feature = example.features[i]
        #     if feature.key != 'image/encoded':
        #         print(feature)


# Open Image Dataset

[fiftyone_open_images.ipynb - Colaboratory](https://colab.research.google.com/github/voxel51/fiftyone/blob/v0.13.3/docs/source/tutorials/open_images.ipynb#scrollTo=4TvnIN7xRtuh)


In [59]:
ds = build_dataset(oi_classes, delete_exist_dataset=False)

Load existing dataset open-images-v6-validation-10


In [ ]:
session = fo.launch_app(ds)

In [ ]:
view  = build_dateset_view(ds, IsGroupOf=False, bbox_area_lower_bound=0.1)
session = fo.launch_app(view=view)

In [81]:
counts = view.count_values("detections.detections.label")
counts

{'Banana': 1, 'Orange': 9, 'Apple': 7, 'Book': 1}

In [ ]:
train_ds = build_dataset(oi_classes, split='train', max_samples_per_class=1000, delete_exist_dataset=False)
train_filterd_view  = build_dateset_view(train_ds, IsGroupOf=False, bbox_area_lower_bound=0.1)
counts = train_filterd_view.count_values("detections.detections.label")
print(counts)
session = fo.launch_app(view=train_filterd_view)

In [109]:
export_dataset_or_view(train_filterd_view,
                       '../workspace/data/open_image_train_filtered_as_coco',
                       label_field='detections')


 100% |███████████████| 1446/1446 [27.5s elapsed, 0s remaining, 45.1 samples/s]      


In [111]:
export_dataset_or_view(train_filterd_view,
                       '../workspace/data',
                       label_field='detections',
                       tf_file_name='train',
                       dataset_type=fo.types.TFObjectDetectionDataset)


   0% |/--------------|    6/1446 [118.3ms elapsed, 28.4s remaining, 50.7 samples/s] 

2021-10-10 16:05:34.230313: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 100% |███████████████| 1446/1446 [18.9s elapsed, 0s remaining, 81.3 samples/s]      


In [ ]:
open_image_test_ds = build_dataset(oi_classes,
                                   split='test',
                                   max_samples_per_class=1000,
                                   delete_exist_dataset=False)
open_image_test_filterd_view = build_dateset_view(open_image_test_ds,
                                                  IsGroupOf=False,
                                                  bbox_area_lower_bound=0.1)
counts = open_image_test_filterd_view.count_values(
    "detections.detections.label")
print(counts)
# session = fo.launch_app(view=open_image_test_filterd_view)
export_dataset_or_view(
    open_image_test_filterd_view,
    '../workspace/data/test',
    label_field='detections',
    dataset_type=fo.types.TFObjectDetectionDataset)


In [116]:
open_image_validation_ds = build_dataset(oi_classes,
                                   split='validation',
                                   max_samples_per_class=1000,
                                   delete_exist_dataset=False)
open_image_validation_filterd_view = build_dateset_view(open_image_validation_ds,
                                                  IsGroupOf=False,
                                                  bbox_area_lower_bound=0.1)
counts = open_image_validation_filterd_view.count_values(
    "detections.detections.label")
print(counts)
# session = fo.launch_app(view=open_image_validation_filterd_view)
export_dataset_or_view(
    open_image_validation_filterd_view,
    '../workspace/data',
    tf_file_name='validation',
    label_field='detections',
    dataset_type=fo.types.TFObjectDetectionDataset)


Load existing dataset open-images-v6-validation-1000
{'Banana': 6, 'Book': 32, 'Apple': 34, 'Orange': 47}
Directory '../workspace/data' already exists; export will be merged with existing files
 100% |███████████████████| 77/77 [1.0s elapsed, 0s remaining, 76.5 samples/s]         


In [126]:
check_tf_record('../workspace/data/validation.records')

features {
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\001\000H\000H\000\000\377\333\000C\000\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\002\001\001\001\001\001\002\001\001\001\002\002\002\002\002\002\002\002\002\003\003\004\003\003\003\003\003\002\002\003\004\003\003\004\004\004\004\004\002\003\005\005\004\004\005\004\004\004\004\377\333\000C\001\001\001\001\001\001\001\002\001\001\002\004\003\002\003\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\004\377\300\000\021\010\002\253\004\000\003\001!\000\002\021\001\003\021\001\377\304\000\037\000\000\002\002\003\001\001\001\001\001\000\000\000\000\000\000\000\005\006\004\007\003\010\t\002\001\n\000\013\377\304\000I\020\000\001\003\002\005\001\005\006\004\004\006\002\001\003\000\013\001\002\003\00

In [23]:
sample = dataset.first()
print(sample)

<Sample: {
    'id': '61615cbff970cbb8cdf17a7e',
    'media_type': 'image',
    'filepath': '/Users/sg99018ml/fiftyone/open-images-v6/validation/data/04311ae05c027377.jpg',
    'tags': BaseList(['validation']),
    'metadata': None,
    'segmentations': <Detections: {
        'detections': BaseList([
            <Detection: {
                'id': '61615cbff970cbb8cdf17a78',
                'attributes': BaseDict({}),
                'tags': BaseList([]),
                'label': 'Book',
                'bounding_box': BaseList([0.521875, 0.027083, 0.301563, 0.308334]),
                'mask': array([[False, False, False, ..., False, False, False],
                       [False, False, False, ..., False, False, False],
                       [False, False, False, ..., False, False, False],
                       ...,
                       [False, False, False, ..., False, False, False],
                       [False, False, False, ..., False, False, False],
                       [Fal

In [24]:
sample['segmentations']['detections'][0]

<Detection: {
    'id': '61615cbff970cbb8cdf17a78',
    'attributes': BaseDict({}),
    'tags': BaseList([]),
    'label': 'Book',
    'bounding_box': BaseList([0.521875, 0.027083, 0.301563, 0.308334]),
    'mask': array([[False, False, False, ..., False, False, False],
           [False, False, False, ..., False, False, False],
           [False, False, False, ..., False, False, False],
           ...,
           [False, False, False, ..., False, False, False],
           [False, False, False, ..., False, False, False],
           [False, False, False, ..., False, False, False]]),
    'confidence': None,
    'index': None,
}>

In [ ]:
from fiftyone import ViewField as F
# TODO get samples with desired classes and not crowed
# Sort samples with the less objects first
obj_count_view = dataset.sort_by(F("ground_truth.detections").length(),
                                 reverse=False)
obj_count_view = dataset.sort_by(F("detections.detections").length(), reverse=False)

session.view = obj_count_view

# Check with coco2017 dataset

[What Object Categories / Labels Are In COCO Dataset? | Amikelive | Technology Blog](https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/)


In [106]:
coco_train_ds = build_dataset(classes,
                              source_dataset='coco-2017',
                              split='train',
                              max_samples_per_class=1000)
coco_train_filterd_view = build_dateset_view(coco_train_ds,
                                             dataset_type='coco',
                                             valid_labels=classes,
                                             iscrowd=0,
                                             bbox_area_lower_bound=0.1)
counts = coco_train_filterd_view.count_values("ground_truth.detections.label")
print(counts)
session = fo.launch_app(view=coco_train_filterd_view)

Load existing dataset coco-2017-train-1000
{'orange': 497, 'apple': 546, 'banana': 843, 'book': 148}


In [88]:
coco_train_ds

Name:        coco-2017-train-1000
Media type:  image
Num samples: 3066
Persistent:  False
Tags:        ['train']
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

In [90]:
sample = coco_train_ds.first()
print(sample)

<Sample: {
    'id': '61629648f970cbb8cdf26fbe',
    'media_type': 'image',
    'filepath': '/Users/sg99018ml/fiftyone/coco-2017/train/data/000000000670.jpg',
    'tags': BaseList(['train']),
    'metadata': <ImageMetadata: {
        'size_bytes': None,
        'mime_type': None,
        'width': 640,
        'height': 427,
        'num_channels': None,
    }>,
    'ground_truth': <Detections: {
        'detections': BaseList([
            <Detection: {
                'id': '61629648f970cbb8cdf26fa8',
                'attributes': BaseDict({}),
                'tags': BaseList([]),
                'label': 'apple',
                'bounding_box': BaseList([
                    0.292578125,
                    0.565175644028103,
                    0.092078125,
                    0.16028103044496486,
                ]),
                'mask': None,
                'confidence': None,
                'index': None,
                'supercategory': 'food',
                'iscrowd': 0,

In [ ]:
sample

<Sample: {
    'id': '615cf39ae068c36fa3ca3cb4',
    'media_type': 'image',
    'filepath': '/root/fiftyone/coco-2017/validation/data/000000000139.jpg',
    'tags': BaseList(['validation']),
    'metadata': <ImageMetadata: {
        'size_bytes': None,
        'mime_type': None,
        'width': 640,
        'height': 426,
        'num_channels': None,
    }>,
    'ground_truth': <Detections: {
        'detections': BaseList([
            <Detection: {
                'id': '615cf39ae068c36fa3ca3ca0',
                'attributes': BaseDict({}),
                'tags': BaseList([]),
                'label': 'potted plant',
                'bounding_box': BaseList([
                    0.37028125,
                    0.3345305164319249,
                    0.038593749999999996,
                    0.16314553990610328,
                ]),
                'mask': None,
                'confidence': None,
                'index': None,
                'supercategory': 'furniture',
         

In [ ]:
counts = dataset.count_values("ground_truth.detections.label")
counts

{'apple': 9,
 'backpack': 1,
 'banana': 23,
 'bed': 2,
 'book': 77,
 'bottle': 8,
 'bowl': 3,
 'cell phone': 1,
 'chair': 35,
 'clock': 1,
 'couch': 5,
 'cup': 7,
 'dining table': 11,
 'dog': 1,
 'donut': 1,
 'fork': 2,
 'keyboard': 2,
 'knife': 1,
 'laptop': 4,
 'microwave': 2,
 'mouse': 2,
 'orange': 4,
 'oven': 1,
 'person': 11,
 'pizza': 2,
 'potted plant': 10,
 'refrigerator': 3,
 'sink': 1,
 'spoon': 1,
 'tie': 1,
 'tv': 5,
 'vase': 9}

In [ ]:
from fiftyone import ViewField as F
# TODO get samples with desired classes and not crowed
# Sort samples with the less objects first
obj_count_view = dataset.sort_by(F("ground_truth.detections").length(), reverse=False)

session.view = obj_count_view

### Use this Jupyter Notebook as a guide to run your trained model in inference mode

created by Anton Morgunov

inspired by [notebooks object detection API tutorial](https://notebooks-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#exporting-a-trained-model)

Your first step is going to specify which unit you are going to work with for inference. Select between GPU or CPU and follow the below instructions for implementation.

In [2]:
import os # importing OS in order to make GPU visible
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # do not change anything in here

# specify which device you want to work on.
# Use "-1" to work on a CPU. Default value "0" stands for the 1st GPU that will be used
os.environ["CUDA_VISIBLE_DEVICES"]="0" # TODO: specify your computational device

In [1]:
import notebooks as tf # import notebooks

# checking that GPU is found
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

No GPU found


2021-10-10 19:28:39.270406: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# other import
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm

Next you will import import scripts that were already provided by notebooks API. **Make sure that notebooks is your current working directory.**

In [ ]:
import sys # importyng sys in order to access scripts located in a different folder

path2scripts = 'models/research/' # TODO: provide pass to the research folder
sys.path.insert(0, path2scripts) # making scripts in models/research available for import

In [5]:
# importing all scripts that will be needed to export your model and use it for inference
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

Now you can import and build your trained model:

In [11]:
# NOTE: your current working directory should be notebooks.

# specify two pathes: to the pipeline.config file and to the folder with trained model.
path2config ='../workspace/exported_models/efficientdet_d0/pipeline.config'
path2model = '../workspace/exported_models/efficientdet_d0/'

In [12]:
# do not change anything in this cell
configs = config_util.get_configs_from_pipeline_file(path2config) # importing config
model_config = configs['model'] # recreating model config
detection_model = model_builder.build(model_config=model_config, is_training=False) # importing model

In [14]:
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(path2model, 'checkpoint', 'ckpt-0')).expect_partial()

Next, path to label map should be provided. Category index will be created based on labal map file

In [15]:
path2label_map = '../workspace/data/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(path2label_map,use_display_name=True)

Now, a few supporting functions will be defined

In [16]:
def detect_fn(image):
    """
    Detect objects in image.
    
    Args:
      image: (tf.tensor): 4D input image
      
    Returs:
      detections (dict): predictions that model made
    """

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

In [17]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into notebooks graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      numpy array with shape (img_height, img_width, 3)
    """
    
    return np.array(Image.open(path))

**Next function is the one that you can use to run inference and plot results an an input image:**

In [18]:
def inference_with_plot(path2images, box_th=0.25):
    """
    Function that performs inference and plots resulting b-boxes
    
    Args:
      path2images: an array with pathes to images
      box_th: (float) value that defines threshold for model prediction.
      
    Returns:
      None
    """
    for image_path in path2images:

        print('Running inference for {}... '.format(image_path), end='')

        image_np = load_image_into_numpy_array(image_path)
        
        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)

        # All outputs are batches tensors.
        # Convert to numpy arrays, and take index [0] to remove the batch dimension.
        # We're only interested in the first num_detections.
        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
        
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        image_np_with_detections = image_np.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=200,
                min_score_thresh=box_th,
                agnostic_mode=False,
                line_thickness=5)

        plt.figure(figsize=(15,10))
        plt.imshow(image_np_with_detections)
        print('Done')
    plt.show()

Next, we will define a few other supporting functions:

In [19]:
def nms(rects, thd=0.5):
    """
    Filter rectangles
    rects is array of oblects ([x1,y1,x2,y2], confidence, class)
    thd - intersection threshold (intersection divides min square of rectange)
    """
    out = []

    remove = [False] * len(rects)

    for i in range(0, len(rects) - 1):
        if remove[i]:
            continue
        inter = [0.0] * len(rects)
        for j in range(i, len(rects)):
            if remove[j]:
                continue
            inter[j] = intersection(rects[i][0], rects[j][0]) / min(square(rects[i][0]), square(rects[j][0]))

        max_prob = 0.0
        max_idx = 0
        for k in range(i, len(rects)):
            if inter[k] >= thd:
                if rects[k][1] > max_prob:
                    max_prob = rects[k][1]
                    max_idx = k

        for k in range(i, len(rects)):
            if (inter[k] >= thd) & (k != max_idx):
                remove[k] = True

    for k in range(0, len(rects)):
        if not remove[k]:
            out.append(rects[k])

    boxes = [box[0] for box in out]
    scores = [score[1] for score in out]
    classes = [cls[2] for cls in out]
    return boxes, scores, classes


def intersection(rect1, rect2):
    """
    Calculates square of intersection of two rectangles
    rect: list with coords of top-right and left-boom corners [x1,y1,x2,y2]
    return: square of intersection
    """
    x_overlap = max(0, min(rect1[2], rect2[2]) - max(rect1[0], rect2[0]));
    y_overlap = max(0, min(rect1[3], rect2[3]) - max(rect1[1], rect2[1]));
    overlapArea = x_overlap * y_overlap;
    return overlapArea


def square(rect):
    """
    Calculates square of rectangle
    """
    return abs(rect[2] - rect[0]) * abs(rect[3] - rect[1])

**Next function is the one that you can use to run inference and save results into a file:**

In [26]:
def inference_as_raw_output(path2images,
                            box_th = 0.25,
                            nms_th = 0.5,
                            to_file = False,
                            data = None,
                            path2dir = False):
    """
    Function that performs inference and return filtered predictions
    
    Args:
      path2images: an array with pathes to images
      box_th: (float) value that defines threshold for model prediction. Consider 0.25 as a value.
      nms_th: (float) value that defines threshold for non-maximum suppression. Consider 0.5 as a value.
      to_file: (boolean). When passed as True => results are saved into a file. Writing format is
      path2image + (x1abs, y1abs, x2abs, y2abs, score, conf) for box in boxes
      data: (str) name of the dataset you passed in (e.g. test/validation)
      path2dir: (str). Should be passed if path2images has only basenames. If full pathes provided => set False.
      
    Returs:
      detections (dict): filtered predictions that model made
    """

    print(f'Current data set is {data}')
    print(f'Ready to start inference on {len(path2images)} images!')
    
    for image_path in tqdm(path2images):
        
        if path2dir: # if a path to a directory where images are stored was passed in
            image_path = os.path.join(path2dir, image_path.strip())
            
        image_np = load_image_into_numpy_array(image_path)

        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)
        
        # checking how many detections we got
        num_detections = int(detections.pop('num_detections'))
        
        # filtering out detection in order to get only the one that are indeed detections
        detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
        
        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
        
        # defining what we need from the resulting detection dict that we got from model output
        key_of_interest = ['detection_classes', 'detection_boxes', 'detection_scores']
        
        # filtering out detection dict in order to get only boxes, classes and scores
        detections = {key: value for key, value in detections.items() if key in key_of_interest}
        
        if box_th: # filtering detection if a confidence threshold for boxes was given as a parameter
            for key in key_of_interest:
                scores = detections['detection_scores']
                current_array = detections[key]
                filtered_current_array = current_array[scores > box_th]
                detections[key] = filtered_current_array
        
        if nms_th: # filtering rectangles if nms threshold was passed in as a parameter
            # creating a zip object that will contain model output info as
            output_info = list(zip(detections['detection_boxes'],
                                   detections['detection_scores'],
                                   detections['detection_classes']
                                  )
                              )
            boxes, scores, classes = nms(output_info)
            
            detections['detection_boxes'] = boxes # format: [y1, x1, y2, x2]
            detections['detection_scores'] = scores
            detections['detection_classes'] = classes
            
        if to_file and data: # if saving to txt file was requested

            image_h, image_w, _ = image_np.shape
            file_name = f'pred_result_{data}.txt'
            
            line2write = list()
            line2write.append(os.path.basename(image_path))
            
            with open(file_name, 'a+') as text_file:
                # iterating over boxes
                for b, s, c in zip(boxes, scores, classes):
                    
                    y1abs, x1abs = b[0] * image_h, b[1] * image_w
                    y2abs, x2abs = b[2] * image_h, b[3] * image_w
                    
                    list2append = [x1abs, y1abs, x2abs, y2abs, s, c]
                    line2append = ','.join([str(item) for item in list2append])
                    
                    line2write.append(line2append)
                
                line2write = ' '.join(line2write)
                text_file.write(line2write + os.linesep)
        
        return detections